## Please select epsic 3.10 env for this notebook 

In [35]:
pwd

'/dls/science/groups/e02/Mohsen/code/jupyterhub_active/Lazarov_analysis_notebooks'

In [ ]:
%matplotlib notebook
import hyperspy.api as hs
import numpy as np
import h5py
import matplotlib.pyplot as plt
from scipy import ndimage
import os
import gc

In [ ]:
d = hs.load('/dls/e01/data/2022/mg30474-1/EDX/AA26/Sample/00_View000/View000_0000004.pts', sum_frames=False, lazy = True)

In [ ]:
d.axes_manager

## Loading the raw data

In [ ]:
zspy_path = '/dls/e01/data/2022/mg31258-11/processing/AH31_EDX_data/AH31_Sample_00_View004_View004_0000411/AH31_Sample_00_View004_View004_0000411.zspy'
hdf_path = '/dls/e01/data/2022/mg31258-11/processing/AH31_EDX_data/AH31_Sample_00_View004_View004_0000411/AH31_Sample_00_View004_View004_0000411.hdf5'
d = hs.load(zspy_path, lazy=True)

In [ ]:
d

In [ ]:
d.change_dtype('uint8')

In [ ]:
d

## Adding elements and calibrate

In [ ]:
d.add_elements(['O', 'Sr', 'Ni', 'Ti', 'Mg', 'Cu', 'C'])

In [ ]:
d.axes_manager

## Plotting sum total spectrum and checking calibration

In [ ]:
d_sum = d.sum(axis=(0,1,2))
d_sum.compute()

In [ ]:
d_sum.plot(True)

In [ ]:
d.metadata

## Get the image stack

In [ ]:
with h5py.File(hdf_path, 'r') as f:
    print(f.keys())
    ims = f['img_frames'][()]
    shifts = f['shifts'][()]

In [ ]:
ims.shape

**shifts** saved here are a rough alignment done during data conversion step. You can either use these shift values or try to align the image stack better and use your refined shift values. Here we are using these shifts to align the spectrum images.

In [ ]:
plt.figure()
plt.plot(shifts)

In [ ]:
ims = hs.signals.Signal2D(ims)
print(ims)
ims.plot()

In [ ]:
ims.estimate_shift2D()

In [ ]:
ims_init = ims.deepcopy()

In [ ]:
ims.align2D(shifts=shifts, crop = False)

In [ ]:
ims.plot()

## Applying shifts - test

This is just to test the align functions below on the images to show that the outcome is the same as above. We then use the same functions to align the SI.

In [ ]:
def shift_image(im, shift=0, interpolation_order=1, fill_value=np.nan):
    if not np.any(shift):
        return im
    else:
        fractional, integral = np.modf(shift)
        if fractional.any():
            order = interpolation_order
        else:
            # Disable interpolation
            order = 0
        return ndimage.shift(im, shift, cval=fill_value, order=order)
    

def shift_si(si, shift):
    """
    si is a hyperspy EDX object
    """
    from functools import partial
    mapfunc = partial(shift_image, shift=shift)
    si_t = si.T
    si_shift = map(mapfunc, si_t.data)
    si_shift = list(si_shift)
    si_shift = np.asarray(si_shift)
    si_shift = si_shift.astype('uint8')
    return hs.signals.Signal2D(si_shift)

In [ ]:
# Test func above on non-aligned stack
with h5py.File(hdf_path, 'r') as f:
    ims = f['img_frames'][()]
    
cent = ims.data.shape[1]//2
ims_crop_shift = []
for i in range(shifts.shape[0]):
    frame_crop = shift_image(ims[i,:,:], shift = list(-1 * shifts[i]))
    ims_crop_shift.append(frame_crop)
ims_crop_shift = np.array(ims_crop_shift)
ims_crop_shift = hs.signals.Signal2D(ims_crop_shift)

In [ ]:
ims_crop_shift.data.shape

In [ ]:
ims_crop_shift.plot()

In [ ]:
d.axes_manager

In [ ]:
frames_num = d.axes_manager[2].size
binned_eds = d.rebin(scale=(4,4,1,2))
elements_list = sorted(['Sr', 'Ni', 'Cu', 'Ti', 'O', 'C'])
lines_list = ['C_Ka', 'Cu_Ka', 'Ni_Ka', 'Ti_Ka' 'O_Ka', 'Sr_Ka']
binned_eds.add_elements(elements_list)
binned_eds.add_lines()

In [ ]:
binned_eds.axes_manager

In [ ]:
binned_eds

In [ ]:
binned_eds.axes_manager

In [ ]:
# Aligning the spectrum images with the same function above

eds_si_aligned = np.zeros((128,128,2048))

# summin g over the central 64 by 64 pixels in each aligend SI

for i in range(frames_num):
    si = binned_eds.inav[:,:,i]
    si.compute()
      si_shifted = shift_si(si, list(-1 * shifts[i]))                          
    print(si_shifted)
    eds_si_aligned += si_shifted.T.data
    
    del si
    gc.collect()

In [ ]:
eds_si_aligned.shape

In [ ]:
eds_si_aligned_hs = hs.signals.EDSTEMSpectrum(eds_si_aligned)
eds_si_aligned_hs

In [ ]:
eds_si_aligned_hs.axes_manager

In [ ]:
binned_eds.axes_manager

In [ ]:
eds_si_aligned_hs.axes_manager[-1].name = 'Energy'
eds_si_aligned_hs.axes_manager[-1].offset = binned_eds.axes_manager[-1].offset
eds_si_aligned_hs.axes_manager[-1].scale = binned_eds.axes_manager[-1].scale
eds_si_aligned_hs.axes_manager[-1].units = binned_eds.axes_manager[-1].units
eds_si_aligned_hs.axes_manager[0].name = binned_eds.axes_manager[0].name
eds_si_aligned_hs.axes_manager[1].name = binned_eds.axes_manager[1].name
eds_si_aligned_hs.axes_manager[0].offset = binned_eds.axes_manager[0].offset
eds_si_aligned_hs.axes_manager[1].offset = binned_eds.axes_manager[1].offset
eds_si_aligned_hs.axes_manager[0].scale = binned_eds.axes_manager[0].scale
eds_si_aligned_hs.axes_manager[1].scale = binned_eds.axes_manager[1].scale
eds_si_aligned_hs.axes_manager[0].units = binned_eds.axes_manager[0].units
eds_si_aligned_hs.axes_manager[1].units = binned_eds.axes_manager[1].units
eds_si_aligned_hs.add_elements(elements_list)
eds_si_aligned_hs.axes_manager

In [ ]:
eds_si_aligned_hs.metadata

In [ ]:
eds_si_aligned_hs.save('/dls/e01/data/2022/mg31258-11/processing/AH31_EDX_data/AH31_Sample_00_View004_View004_0000411/Aligned_SI')

In [ ]:
maps = eds_si_aligned_hs.get_lines_intensity()

In [ ]:
hs.plot.plot_images(maps, cmap='inferno')